In [17]:
from pathlib import Path
from io import StringIO
import os
import json

import girder_client

class DsaExporter:
    def __init__(self, token, root_id, root_type='collection', host='0.0.0.0', port=8080):
        # sudo ip addr show docker0 | grep inet | grep docker0 | awk '{print $2}' | sed 's/\/.*//'
        # will give you the ip address of the docker network which binds to localhost
        client = girder_client.GirderClient(apiUrl=f'http://{host}:{port}/api/v1')
        client.authenticate(apiKey=token)
        self._client = client
        self._root_id = root_id
        self._root_type = root_type
        
    def _export_file(self, filepath, parent_id, metadata={}):
        filepath = Path(filepath)
#         parent_id = self._makedirs(filepath)
        response = self._client.createItem(
            parent_id,
            filepath.name,
            metadata=metadata,
        )
        parent_id = response['_id']
        response = self._client.uploadFileToItem(parent_id, filepath)
        return response['_id']

    def _export_folder(self, dirpath, metadata={}):
        dirpath = Path(dirpath)
        parent_id = self._makedirs(dirpath)
        response = self._client.createFolder(
            parent_id,
            dirpath.name,
            metadata=metadata,
            reuseExisting=True
        )
        return response['_id']
    
    def _makedirs(self, dirpath):
        dirpath = Path(dirpath)
        folders = reversed(dirpath.parents)
        folders = [x.name for x in folders]
        folders = filter(lambda x: x != '', folders)

        parent_id = self._root_id
        parent_type = self._root_type
        for folder in folders:
            response = self._client.createFolder(
                parent_id,
                folder,
                reuseExisting=True,
                parentType=parent_type
            )
            parent_id = response['_id']
            parent_type = 'folder'
        return parent_id
    
    def export(self, root):
        data = Path(root, 'data')
        meta = Path(root, 'metadata')
        assets = Path(meta, 'asset')
        files = Path(meta, 'file')
        for path in [data, meta, assets, files]:
            if not path.is_dir():
                msg = f'{path.as_posix()} directory does not exist.'
                raise FileNotFoundError(msg)
        # ---------------------------------------------------------------
                
        for asset in os.listdir(assets):
            asset = Path(assets, asset)
            with open(asset) as f:
                asset = json.load(f)
            
            if asset['asset_type'] == 'file':
                parent_id = self._export_folder(asset['asset_path_relative'])
                self._export_file(asset['asset_path'], metadata=asset)

            else:
                self._export_folder(asset['asset_path_relative'], metadata=asset)

                filepaths = asset['file_ids']
                filepaths = [Path(files, f'{x}.json') for x in filepaths]
                for filepath in filepaths:
                    with open(filepath, 'rb') as f:
                        metadata = json.load(f)
                    filepath = metadata['filepath']
                    parent_id = self._makedirs(metadata['filepath_relative'])
                    self._export_file(filepath, parent_id, metadata=metadata)

        
token = 'eyS0nj9qPC5E7yK5l7nhGVPqDOBKPdA3EC60Rs9h'
host = '172.17.0.1'
root = '5ed735c8d8dd6242642406e5'
uploader = DsaExporter(token, root, host=host)
uploader.export('/tmp/hidebound')
# filepath = '../resources/test_config.json'
# metadata = dict(alligator=1, bagel=2, crayon=3, cat='john', dog=54, foo=dict(bar='baz'))
# uploader._export_file(filepath, pid, metadata)

p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002
p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0001.jpg
p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0002.jpg
p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v002/p-cat001_s-spec002_d-tabby-playing_v002_f0003.jpg
p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001
p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0001.jpg
p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0002.jpg
p-cat001/s-spec002/d-calico-jumping/p-cat001_s-spec002_d-calico-jumping_v001/p-cat001_s-spec002_d-calico-jumping_v001_f0003.jpg
p-cat001/s-spec002/d-tabby-playing/p-cat001_s-spec002_d-tabby-playing_v001
p-cat001/s-spe